In [1]:
from googleapiclient.discovery import build
from IPython.display import JSON, display
import re
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import os
# from git import Repo
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
# import zipfile
import requests


In [2]:
api_key = user_secrets.get_secret("YouTubeAPIKeyTwo")
api_service_name = "youtube"
api_version = "v3"
youtube = build(api_service_name, api_version, developerKey=api_key)

In [3]:
kw_list =  ['donut']

In [4]:

def VideoDetailExtraction(publishedAfter,maxResults = 5):
    request = youtube.search().list(part = 'snippet',
                                    order = 'viewCount',
                                    q = kw_list,
                                    relevanceLanguage = 'en',
                                    type = 'video',
                                    videoCategoryId = 26,
                                    regionCode = 'US',
                                    maxResults =5,
                                    publishedAfter = publishedAfter)
    response = request.execute()
    videoDetails = []
    videoIds = []
    for i in range(0,maxResults):
        publishTime = re.split(r'[TZ-]',response['items'][0]['snippet']['publishTime'])

        videoDetails.append({'videoId' : response['items'][i]['id']['videoId'],
            'channelId' : response['items'][i]['snippet']['channelId'],
            'channelName' : response['items'][i]['snippet']['channelTitle'],
            'videoTitle' : response['items'][i]['snippet']['title'],
            'description' : response['items'][i]['snippet']['description'],
            'publishYear' : publishTime[0], #year
            'publishMonth' : publishTime[1], #month
            'publishDay' : publishTime[2], #day
            'publishTime' : publishTime[3], #hh:mm:ss
            'publishedOn' : response['items'][0]['snippet']['publishTime']
        })

        videoIds.append(response['items'][i]['id']['videoId'])

    
    request = youtube.videos().list(
        part = 'statistics,snippet', 
        id = videoIds
    )
    response = request.execute()
    JSON(response)
    for i in range(0,maxResults):
        videoDetails[i]['viewCount'] = response['items'][i]['statistics'].get('viewCount',0)
        videoDetails[i]['likeCount'] = response['items'][i]['statistics'].get('likeCount',0)
        videoDetails[i]['favoriteCount'] = response['items'][i]['statistics'].get('favoriteCount',0)
        videoDetails[i]['commentCount'] = response['items'][i]['statistics'].get('commentCount',0)
        videoDetails[i]['videoTags'] = response['items'][i]['snippet'].get('tags',[])
    # display(videoDetails)
    videoDetails = pd.DataFrame(videoDetails)
    return videoDetails

In [5]:
def PushToGithub():
    output_files = os.listdir('/kaggle/working')
    # print("Files in /kaggle/working:", output_files)
    # repo_url = user_secrets.get_secret("Repo_URL_YouTubeFood")
    Today = max([file for file in output_files if file.startswith("Today")], 
                key=os.path.getctime)
    Last30Days = max([file for file in output_files if file.startswith("Last30Days")],
                     key = os.path.getctime)
    Last1Year = max([file for file in output_files if file.startswith("Last1Year")],
                    key = os.path.getctime)
    Last5Year = max([file for file in output_files if file.startswith("Last5Year")],
                    key = os.path.getctime)
    # print(Today,Last30Days,Last1Year,Last5Year) 
   
    token = user_secrets.get_secret("GITHUB_TOKEN")
    owner = user_secrets.get_secret("REPO_OWNER")
    nameOfRepo = user_secrets.get_secret("REPO_NAME")
    API_URL = f"https://api.github.com/repos/{owner}/{nameOfRepo}/contents"
    headers = {"Authorization": f"token {token}"}
    response = requests.get(API_URL, headers=headers)
    print(response)


    

In [6]:
def RawFile(dataframe,prefix):
    # Generate a timestamped filename
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    filename = f"{prefix}_{timestamp}.json"

    # Save the DataFrame as a JSON file
    dataframe.to_json(filename, orient="records", indent=4)
    print(f"DataFrame saved as {filename}")
    return True

In [7]:
def TopVideosBasedOnTime():
    currentDate = datetime.date.today()
    #today
    today = "{}T00:00:00Z".format(currentDate)
    today_response = VideoDetailExtraction(today,1)
    RawFile(today_response,'Today')
    #Last 30 days
    Last30Days = currentDate + relativedelta(days=-30)
    Last30Days = "{}T00:00:00Z".format(Last30Days)
    Last30Days_response = VideoDetailExtraction(Last30Days,1)
    RawFile(today_response,'Last30Days')
    #Last 1 year
    Last1Year = currentDate + relativedelta(years = -1)
    Last1Year = "{}T00:00:00Z".format(Last1Year)
    Last1Year_response = VideoDetailExtraction(Last1Year,1)
    RawFile(today_response,'Last1Year')
    #Last 5 years
    Last5Years = currentDate + relativedelta(years = -5)
    Last5Years = "{}T00:00:00Z".format(Last5Years)
    Last5Years_response = VideoDetailExtraction(Last5Years,1)
    RawFile(today_response,'Last5Years')
    
    return True

TopVideosBasedOnTime()

DataFrame saved as Today_2025-01-17_15:47:42.json
DataFrame saved as Last30Days_2025-01-17_15:47:43.json
DataFrame saved as Last1Year_2025-01-17_15:47:43.json
DataFrame saved as Last5Years_2025-01-17_15:47:43.json


True

In [8]:
def main():
    TopVideosBasedOnTime()

In [9]:
# if __name__=="__main__":
#     main()